In [9]:
import tensorflow as tf 
from tensorflow.keras import layers, models

import pathlib
import os

In [10]:
import deeplake
import numpy as np

In [11]:
# Loads plant_village dataset
ds = deeplake.load('hub://activeloop/plantvillage-without-augmentation')

|

Opening dataset in read-only mode as you don't have write permissions.


|

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/plantvillage-without-augmentation



\

hub://activeloop/plantvillage-without-augmentation loaded successfully.



In [12]:
# Prints shape of plant_village images after turning it into np.ndarray format
# This cell takes five minutes to run (rip)
img = ds.images.numpy()
print(np.shape(img))
print(type(img))


MemoryError: Unable to allocate 9.67 GiB for an array with shape (52803, 256, 256, 3) and data type uint8

In [13]:
# Prints shape of plant_village labels
label = ds.labels.numpy(aslist=True)
labels = np.array(label)

print(labels.shape)
print(labels[:10])

(52803, 1)
[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]


In [14]:
# Clones PlantDoc dataset from GitHub into Google Colab
# This cell could take a few minutes
# print(os.getcwd())
!git clone https://github.com/pratikkayal/PlantDoc-Dataset.git

Cloning into 'PlantDoc-Dataset'...
error: invalid path 'test/Bell_pepper leaf/IMG_1629.JPG?1507122477.jpg'
fatal: unable to checkout working tree
You can inspect what was checked out with 'git status'
and retry with 'git restore --source=HEAD :/'



fatal: not a git repository (or any of the parent directories): .git


In [20]:
# Access PlantDoc training dataset
# print(os.getcwd())
data_dir = pathlib.Path("PlantDoc-Dataset/train").with_suffix("")

image_count_train = len(list(data_dir.glob('*/*.jpg')))

batch_size = 32
img_height = 256
img_width = 256

train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

# class_names = train_ds.class_names
# print(class_names)

# Access PlantDoc testing dataset
data_dir = pathlib.Path("PlantDoc-Dataset/test").with_suffix("")
image_count_test = len(list(data_dir.glob('*/*.jpg')))

test_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 0 files belonging to 1 classes.


Using 0 files for training.


ValueError: No images found in directory PlantDoc-Dataset. Allowed formats: ('.bmp', '.gif', '.jpeg', '.jpg', '.png')

In [ ]:
# Convert data into np.ndarray format

plantDoc_images = np.ndarray([1, 256, 256, 3])
plantDoc_labels = np.ndarray([1])
plantDoc_images_test = np.ndarray([1, 256, 256, 3])
plantDoc_labels_test = np.ndarray([1])

count = 0
for image_batch, labels_batch in train_ds:
#   if count % 5 == 0:
#     print("Batch number", count)
  plantDoc_images = np.append(plantDoc_images, image_batch.numpy(), axis = 0)
  plantDoc_labels = np.append(plantDoc_labels, labels_batch.numpy(), axis = 0)
  count += 1

print(plantDoc_images.shape, plantDoc_labels.shape)

count = 0
for image_batch, labels_batch in test_ds:
#   if count % 5 == 0:
#     print("Batch number", count)
  plantDoc_images_test = np.append(plantDoc_images_test, image_batch.numpy(), axis = 0)
  plantDoc_labels_test = np.append(plantDoc_labels_test, labels_batch.numpy(), axis = 0)
  count += 1

print(plantDoc_images_test.shape, plantDoc_labels_test.shape)

In [20]:
# Shuffle data
np.random.shuffle(img)
np.random.shuffle(labels)

np.random.shuffle(plantDoc_images)
np.random.shuffle(plantDoc_labels)
np.random.shuffle(plantDoc_images_test)
np.random.shuffle(plantDoc_labels_test)

In [ ]:
# Concatenate datasets
train_images = np.append(img[0:40727], plantDoc_images, axis = 0)
test_images = np.append(img[40728:52803], plantDoc_images_test, axis = 0)

train_labels = np.append(labels[0:40727], plantDoc_labels, axis = 0)
test_labels = np.append(labels[40728:52803], plantDoc_labels_test, axis = 0)

In [ ]:
# Save data to files
with open("train_images.pkl", mode="wb") as opened_file:
    pickle.dump(train_images, opened_file)
with open("test_images.pkl", mode="wb") as opened_file:
    pickle.dump(test_images, opened_file)
with open("train_labels.pkl", mode="wb") as opened_file:
    pickle.dump(train_labels, opened_file)
with open("test_labels.pkl", mode="wb") as opened_file:
    pickle.dump(test_labels, opened_file)